In [1]:
%cd C:\Users\yukir\Documents\GitHub\Text_Mining\py

from nlp_processing import *
set_global_determinism(seed=1337)

%cd C:\Users\yukir\Documents\Monicas_workspace\Derma

C:\Users\yukir\Documents\GitHub\Text_Mining\py
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6090800915783322101
]
tf 2.5.3
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라


C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


C:\Users\yukir\Documents\Monicas_workspace\Derma


In [2]:
import openpyxl
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import matplotlib.pyplot as plt

from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis

from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm 
import re 
import pickle 
import csv

from hanspell import spell_checker

from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

In [9]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

In [7]:
org_df = pd.read_excel("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma\\[0327]final_derma.xlsx", index_col = 0)
data = org_df.copy()
print(data.shape)
data.head()

(39216, 13)


,h,adr,d,f,reviews,stars,treatments,money,비고,do,si,ro,adr_else
0,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 3월 15일, 2022년 게시 ...",두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,두피 가려움증,4900,리뷰없음,경남,김해시,내외중앙로,57
1,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 12월 20일, 2021년 게시 ...",대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,대상포진,4200,리뷰없음,경남,김해시,내외중앙로,57
2,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 8월 11일, 2021년 게시 ...",여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,관리 안됨,4000,리뷰없음,경남,김해시,내외중앙로,57
3,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),있음,"^^있다^^영수증 인증 6월 21일, 2021년 게시 ...",문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,문신제거,22000,리뷰없음,경남,김해시,내외중앙로,57
4,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 6월 7일, 2021년 게시 ...",원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,피부트러블,4000,리뷰없음,경남,김해시,내외중앙로,57


In [8]:
new_treat = []

for i in tqdm(data['treatments'].tolist()):
    try:
        k = clean_text(i)
        k = spell_checker.check(k)
        k = k.checked
        new_treat.append(k)
    except:
        new_treat.append(k)

data['treatments'] = new_treat

100%|██████████| 39216/39216 [21:09<00:00, 30.89it/s]  


In [12]:
len(data['treatments'].unique())

10712

In [13]:
pd.DataFrame(data['treatments'].value_counts()).to_excel('[0501]treatments.xlsx')

In [28]:
tmt = pd.read_excel("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma\\[0501]treatments.xlsx")
tmt.head()

,No,treatment,count,leave
0,1,염증 주사,2079,NaN
1,2,여드름 약물치료,1162,NaN
2,3,점 비립종 쥐젖 제거 레이저,849,NaN
3,4,피부염,782,NaN
4,5,두드러기,765,NaN


In [30]:
tmt_list = tmt[tmt['leave']=='삭제']['treatment'].tolist()

In [35]:
print(data.shape)
data_v2 = data[~data['treatments'].isin(set(tmt_list))]
print(data_v2.shape)

(39216, 13)
(37087, 13)


,h,adr,reviews,stars,treatments,money,adr_else
0,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,두피 가려움증,4900,57
1,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,대상포진,4200,57
2,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,관리 안 됨,4000,57
3,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,문신제거,22000,57
4,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,피부 트러블,4000,57


In [90]:
from datetime import datetime
import datetime
from datetime import timedelta

def str_to_dt(x):
    x = str(x)
    y = x[0:4]
    m = x[4:6]
    d = x[6:8]
    format_ = '%Y-%m-%d'
    # date = y+'-'+m+'-'+d
    dates = datetime.datetime.strptime(dates, format_)
    return dates

In [141]:
format_ = '%Y-%m-%d'
# dates = datetime.datetime.strptime(date, format_)
dates_list_v = []
for i in data_v2.f.tolist():
    try: 

        ## reviews
        reviews = i.split('                    ')[0]
        reviews = reviews.split('                ')[-1]
        reviews = reviews.split('받은 치료 :            ')[-1]

        ## dates
        dates = i.split('게시')[0]
        dates = dates.split('  ')[-1]
        y = dates.split(',')[-1]
        y = y.replace('년 ','')
        y = y.strip()
        dates = dates.split(',')[0]
        dates = dates.replace('일','')
        dates = dates.replace('월 ','-')
        m = dates.split('-')[0]
        if len(m) != 2:
            m = '0'+m
        d = dates.split('-')[1]
        if len(d) != 2:
            d = '0'+d
            # dates = datetime.datetime.strptime(y+'-'+m+'-'+d,format_)
            # dates = str_to_dt(dates)
            # ## 치료가격
            # cost = i.split('치료 가격')[-1]
            # cost = cost.split('원 ')[0]
            # cost = cost.split('원')[0]
            # cost = cost.split('정보')[-1]


            # dates_list.append(dates)
        date_str = y+'-'+m+'-'+d
        dates = datetime.datetime.strptime(date_str, format_)
        dates_list_v.append(dates)
    except:
        dates_list_v.append(i)
#     dates_list_v.apppend(date_str)
   

data_v2['dates'] = dates_list_v

C:\Users\yukir\AppData\Local\Temp/ipykernel_20812/3702328806.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_v2['dates'] = dates_list_v


In [143]:
data_v2.head()

,h,adr,d,f,reviews,stars,treatments,money,비고,do,si,ro,adr_else,dates
0,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 3월 15일, 2022년 게시 ...",두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,두피 가려움증,4900,리뷰없음,경남,김해시,내외중앙로,57,2022-03-15 00:00:00
1,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 12월 20일, 2021년 게시 ...",대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,대상포진,4200,리뷰없음,경남,김해시,내외중앙로,57,2021-12-20 00:00:00
2,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 8월 11일, 2021년 게시 ...",여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,관리 안 됨,4000,리뷰없음,경남,김해시,내외중앙로,57,2021-08-11 00:00:00
3,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),있음,"^^있다^^영수증 인증 6월 21일, 2021년 게시 ...",문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,문신제거,22000,리뷰없음,경남,김해시,내외중앙로,57,2021-06-21 00:00:00
4,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 6월 7일, 2021년 게시 ...",원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,피부 트러블,4000,리뷰없음,경남,김해시,내외중앙로,57,2021-06-07 00:00:00


In [144]:
data_v2 = data_v2[['h','adr','reviews', 'stars', 'treatments', 'money', 'adr_else','dates']]
data_v2.head()

,h,adr,reviews,stars,treatments,money,adr_else,dates
0,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,두피 가려움증,4900,57,2022-03-15 00:00:00
1,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,대상포진,4200,57,2021-12-20 00:00:00
2,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,관리 안 됨,4000,57,2021-08-11 00:00:00
3,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,문신제거,22000,57,2021-06-21 00:00:00
4,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,피부 트러블,4000,57,2021-06-07 00:00:00


In [145]:
data_v2.to_excel('[0501]final_reprocessed_derma.xlsx')

In [4]:
data = pd.read_excel('[0501]final_reprocessed_derma.xlsx', index_col=0)
data.head()

,h,adr,reviews,stars,treatments,money,adr_else,dates
0,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,두피 가려움증,4900,57,2022-03-15 00:00:00
1,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,대상포진,4200,57,2021-12-20 00:00:00
2,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,관리 안 됨,4000,57,2021-08-11 00:00:00
3,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,문신제거,22000,57,2021-06-21 00:00:00
4,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,피부 트러블,4000,57,2021-06-07 00:00:00


In [5]:
from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

In [15]:
def stemming (text):
    morphs_list = []
    one_words = []
    result = []

    pos = okt.pos(text, join = True)
    # print(pos)
    for j in pos:
        if j.split('/')[1] == 'Noun':
            # j = j.split('/')[0]
            morphs_list.append(j)
                    
        elif j.split('/')[1] =='Adjective':
            k = okt.morphs(j,  stem= True)
            # k = k[0]
            morphs_list.append(k)
        elif j.split('/')[1] =='Verb':
            v = okt.morphs(j,  stem= True)
            # v = v[0]
            morphs_list.append(v)

    for i in morphs_list:
        if len(i) != 1:
            one_words.append(i)
    

    for i in one_words:
        if i not in stopwords_kor:
            result.append(i)

    return result

In [25]:
print(data.shape)
data_v2 = data.drop_duplicates()
print(data_v2.shape)
data_v2.head()

(37087, 8)
(29408, 8)


,h,adr,reviews,stars,treatments,money,adr_else,dates
0,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,두피 가려움증,4900,57,2022-03-15 00:00:00
1,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,대상포진,4200,57,2021-12-20 00:00:00
2,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,관리 안 됨,4000,57,2021-08-11 00:00:00
3,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,문신제거,22000,57,2021-06-21 00:00:00
4,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,피부 트러블,4000,57,2021-06-07 00:00:00


In [26]:
cleaned_reviews = []
stemmed = []

for i in tqdm(data_v2.reviews.tolist()):
    k = clean_text(i)
    k = k.replace(' ','')
    k = spell_checker.check(k)
    k = k.checked
    cleaned_reviews.append(k)

    s = okt.pos(k,join=True)
    stemmed.append(s)

  0%|          | 111/29408 [00:57<2:22:20,  3.43it/s]

In [ ]:
data_v2['cleaned_reviews'] = cleaned_reviews
data_v2['stemmed'] = stemmed

In [ ]:
data_v2.to_excel('[0502]final_stemmed_derma.xlsx')